In [1]:
import ipywidgets as widgets
import ipympl
from IPython.display import display
from localpackage.game import game
from localpackage.utils import returnFreq

from bqplot import *
import bqplot.pyplot as plt

import numpy as np
import math
regions=['UK','EW','EN','SC','WA','NI','GB']
lbeq=widgets.Label(value='=',layout=widgets.Layout(display="flex", justify_content="flex-start", width="10px"))
ClaimantOut = widgets.Output(layout=widgets.Layout(display="flex", justify_content="flex-start", width="420px", border='solid'))
DependentOut= widgets.Output(layout=widgets.Layout(display="flex", justify_content="flex-start", width="420px", border='solid')) 

In [2]:
#Game
dr=widgets.FloatSlider(value=-0.5,min=-3, max=3, description='Disc. Rate:')
td=widgets.DatePicker(description='Trial Date:',disabled=False, visible=False)
og=widgets.Dropdown(
    options=['Ogden 7', 'Ogden 8'],
    value='Ogden 8',
    description='Ogden:',
    disabled=False,
)
fatal=widgets.Checkbox(value=False,description='Fatal',disabled=False,indent=False,layout=widgets.Layout(width='100px'))
yrattained=widgets.Checkbox(value=False,description='Auto Year Attained',disabled=False,indent=False,layout=widgets.Layout(width='200px'))
i1=[dr,td,og,fatal,yrattained]
h1=widgets.HBox(i1)
display(h1)
g={'discountRate':dr.value,'Ogden':7}

In [3]:
def on_value_change(change):
    fillResults()

In [4]:
#Person
def persondetails(person):
    label=widgets.Label(value=person.capitalize()+":",layout=widgets.Layout(width='100px'))
    age=widgets.FloatSlider(value=30,min=0,max=125, step=1,description='Age at trial:', layout=widgets.Layout(width='400px'))
    sex=widgets.Dropdown(
        options=['Male', 'Female'],
        value='Male',
        description='Sex:',
        disabled=False,
        layout=widgets.Layout(width='175px')
    )
    rg=widgets.Dropdown(
        options=regions,
        value='UK',
        description='Region:',
        disabled=False,
        layout=widgets.Layout(width='175px'),
    )
    deltale=widgets.FloatText(value=0.0, description='deltaLE:', disabled=False, layout=widgets.Layout(width='150px'))
    ageatdeath=widgets.FloatSlider(value=age.value-5,min=0,max=age.value, step=1,description='Age at death:', layout=widgets.Layout(width='400px'))

    i2=[label,age,sex,rg,deltale,ageatdeath]
    h2=widgets.HBox(i2)
    display(h2)
    return age, sex, rg, deltale, ageatdeath, label, h2

def getvaluesPerson(person):
    person=persons[person]
    return person[0].value, person[1].value, person[2].value, person[3].value, person[4].value, person[5].value

In [5]:
#Create persons
persons={}
persons['CLAIMANT']=persondetails('CLAIMANT')
persons['DEPENDENT']=persondetails('DEPENDENT')

In [6]:
#Row person
def rowPerson(person):
    age, sex, region, deltale, ageatdeath, label=getvaluesPerson(person)
    label=widgets.Label(value=person.capitalize()+":",layout=widgets.Layout(width='100px'))
    r=widgets.FloatRangeSlider(
        value=[age, 125],
        min=0,
        max=125,
        step=1,
        description='Age Range:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='400px')
    )
    fr=widgets.Text(value='Y', description='Freq:',layout=widgets.Layout(width='175px'))
    op=widgets.Text(value='AMI', description='Options:',layout=widgets.Layout(width='150px'))
    i4=[label,r,fr,op,lbeq,ClaimantOut]
    h4=widgets.HBox(i4)
    display(h4)
    return r, fr, op, label, h4

def getvaluesRow(person):
    row=rows[person]
    return row[0].value[0],row[0].value[1],row[1].value,row[2].value, row[3].value
    

In [7]:
#Create rows for dependent
rows={}
rows['CLAIMANT']=rowPerson('CLAIMANT')
rows['DEPENDENT']=rowPerson('DEPENDENT')
rows['DEPENDENT'][2].value='AMID'

In [13]:

def getdataSet(og,region):
    if og=='Ogden 7':
        return {'year':2008,'region':region,'yrAttainedIn':2011}
    elif og=='Ogden 8':
        return {'year':2018,'region':region,'yrAttainedIn':2022}
    else:
        #error
        print('Wrong Ogden')

def getGame():
    claimants=[]
    age, sex, region, deltale, ageatdeath, label=getvaluesPerson('CLAIMANT')
    if fatal.value:
        claimant = {'name': 'CLAIMANT', 'age': age,'aad':ageatdeath, 'sex': sex, 'dataSet': getdataSet(og.value,region), 'deltaLE': deltale, 'cont':1}        
    else:
        claimant = {'name': 'CLAIMANT', 'age': age, 'sex': sex, 'dataSet': getdataSet(og.value,region), 'deltaLE': deltale, 'cont':1}

    age, sex, region, deltale, ageatdeath,label=getvaluesPerson('DEPENDENT')
    dependent= {'name': 'DEPENDENT', 'age': age, 'sex': sex, 'dependenton':'CLAIMANT', 'dataSet':getdataSet(og.value,region),'deltaLE': deltale, 'cont':1}
    claimants.append(claimant)
    claimants.append(dependent)
    eg={'game':{'autoYrAttained':yrattained.value,'discountRate':dr.value/100, 'Ogden':7, 'claimants':claimants}}
    return game(eg)

def stringifyResult(result):
    return "Past ("+result[0]+") + Int("+result[1]+") + Future("+result[2]+") = " + result[3]

def getResult(person, thisGame):
    fromAge,toAge, frequency, options, label=getvaluesRow(person)
    if (fromAge==toAge):
        result=thisGame.getClaimant(person).M(fromAge, freq=frequency, options=options)
    else:
        result=thisGame.getClaimant(person).M(fromAge, toAge, freq=frequency, options=options)
    return result    

def agegap():
    ageC, sex, region, deltale, ageatdeath, label=getvaluesPerson('CLAIMANT')
    ageD, sex, region, deltale, ageatdeath, label=getvaluesPerson('DEPENDENT')
    return ageC-ageD

def getLeftX(fat):
    ageC, sex, region, deltale, ageatdeathC, label=getvaluesPerson('CLAIMANT')
    ageD, sex, region, deltale, ageatdeathD, label=getvaluesPerson('DEPENDENT')
    fromAgeC,toAge, frequency, options, label=getvaluesRow('CLAIMANT')
    fromAgeD,toAge, frequency, options, label=getvaluesRow('DEPENDENT')
    if fat: return min(ageC,fromAgeC,ageatdeathC)-1
    return min(ageC,fromAgeC)-1

def getageatDeath():
    age, sex, region, deltale, ageatdeath, label=getvaluesPerson('CLAIMANT')
    return ageatdeath

    
    

def updateBQplot(person, thisGame,Res):
    agap=0
    if person=='DEPENDENT': agap=agegap()
    
    rows['CLAIMANT'][2].value=rows['CLAIMANT'][2].value.upper()
    rows['DEPENDENT'][2].value=rows['DEPENDENT'][2].value.upper()

    age, sex, region, deltale, ageatdeath, label=getvaluesPerson(person)
    fromAge,toAge, frequency, options, label=getvaluesRow(person)
    pdf_fig, pdf_line, pdf_age, pdf_aad, pdf_fill, pdf_start, pdf_end= pdf[person]
    
    
    curve=thisGame.getClaimant(person).getCurve()
    if not fromAge==toAge:
        title=curve.getTitle(result=Res,fromAge=fromAge,toAge=toAge,freq=frequency,cont=1,options=options)        
    else:
        title=curve.getTitle(result=Res,fromAge=fromAge,freq=frequency,cont=1,options=options)

        
    #The plot
    LxNoI,Lx,Rng=curve.getCurve(options=options,cont=1)
    pdf_fig.title=title
    #scale
    leftX=getLeftX(fatal.value)-agap
        
    pdf_line.scales['x'].min=leftX
    pdf_line.scales['x'].max=125-agap
    pdf_line.scales['y'].min=0
    pdf_line.scales['y'].max=2
    pdf_line.x=Rng
    pdf_line.y=Lx
    #age at trial
    pdf_age.x=[age, age]
    pdf_age.y=[0, 1]
    #Age at death
    pdf_aad.x=[getageatDeath()-agap,getageatDeath()-agap]
    pdf_aad.y=[0,1]

    #Sort out fill range
    mask=(Rng>=fromAge) & (Rng<=toAge)
    R=Rng[mask]
    L=Lx[mask]
    #add interp points
    Ll=np.interp(fromAge,Rng,Lx)
    Ul=np.interp(toAge,Rng,Lx)
    R=np.insert(R,0,fromAge)
    L=np.insert(L,0,Ll)
    R=np.append(R,toAge)
    L=np.append(L,Ul)
    
    #single point
    y1=np.interp(fromAge,Rng,Lx)
    pdf_start.x=[fromAge,fromAge]
    pdf_start.y=[0,y1]
    y2=np.interp(toAge,Rng,Lx)
    pdf_end.x=[toAge,toAge]
    pdf_end.y=[0,y2]
    #the fill
    pdf_fill.x=R
    pdf_fill.y=L
    


def fillResults():
    thisGame=getGame()
    with ClaimantOut:
        cRes=getResult('CLAIMANT',thisGame)
        cResStr=["{:2.2f}".format(f) for f in cRes]
        print(stringifyResult(cResStr))
        ClaimantOut.clear_output(wait=True) 
            
    with DependentOut:
        dRes=getResult('DEPENDENT',thisGame)
        dResStr=["{:2.2f}".format(f) for f in dRes]
        print (stringifyResult(dResStr))
        DependentOut.clear_output(wait=True) 
    
    updateBQplot('CLAIMANT',thisGame,cRes)
    if fatal.value: updateBQplot('DEPENDENT',thisGame,dRes)


def setFatal():




    ageC, sexC, regionC, deltaleC, ageatdeathC, labelPC, hPC=persons['CLAIMANT']
    ageD, sexD, regionD, deltaleD, ageatdeathD, labelPD, hPD=persons['DEPENDENT']
    rC, frC, opC, labelRC, h4C=rows['CLAIMANT']
    rD, frD, opD, labelRD, h4D=rows['DEPENDENT']

    
    pdf_figC, pdf_lineC, pdf_ageC, pdf_aadC, pdf_fillC, pdf_startC, pdf_endC= pdf['CLAIMANT']



    if fatal.value:#it's a fatal game
        pdf['DEPENDENT']=drawFigure('DEPENDENT')

        pdf_figD, pdf_lineD, pdf_ageD, pdf_aadD, pdf_fillD, pdf_startD, pdf_endD=pdf['DEPENDENT']

        labelPC.value='Deceased:'
        labelRC.value='Deceased:'
        hPD.layout.display=None
        h4D.layout.display=None
        ageatdeathD.layout.display='none' #hide slider for age at death of D
        ageatdeathC.layout.display=None #show slider for age at death of C
        pdf_aadC.display_legend=True
        pdf_aadD.display_legend=True
        pdf_aadC.visible=True
        pdf_aadD.visible=True
    else:
        plt.figure('DEPENDENT')
        plt.figure('DEPENDENT').title=''
        plt.clear()

        labelPC.value='Claimant:'
        labelRC.value='Claimant:'
        ageatdeathC.layout.display='none' #hide slider for age at death of C
        pdf_aadC.display_legend=False
        pdf_aadC.visible=False
        hPD.layout.display='none'
        h4D.layout.display='none'
        

    

def on_fatal_change(change):
    setFatal()

def on_value_change_age(change):
    ageC, sex, region, deltale, ageatdeathC, label=getvaluesPerson('CLAIMANT')
    a,s,r,d,aad,lbl,h=persons['CLAIMANT']
    if ageatdeathC>ageC:
        aad.value=ageC
    aad.max=ageC
    fillResults()


#SET UP EVENT HANDLERS
[persons['CLAIMANT'][x].observe(on_value_change,names='value') for x in [1,2,3,5,6]]
[persons['DEPENDENT'][x].observe(on_value_change,names='value') for x in range(0,len(persons['DEPENDENT']))]
[rows['CLAIMANT'][x].observe(on_value_change,names='value') for x in range(0,len(rows['CLAIMANT']))]
[rows['DEPENDENT'][x].observe(on_value_change,names='value') for x in range(0,len(rows['DEPENDENT']))]

persons['CLAIMANT'][4].observe(on_value_change_age,names='value') #claimant's age at death
persons['CLAIMANT'][0].observe(on_value_change_age,names='value') #claimant's age

og.observe(on_value_change,names='value')
dr.observe(on_value_change, names='value')
og.observe(on_value_change,names='value')
yrattained.observe(on_value_change,names='value')
fatal.observe(on_fatal_change,names='value')

thisGame=getGame()

LxNoI=Lx=Rng=None


In [9]:
def drawFigure(person):
    curve=thisGame.getClaimant(person).getCurve()
    age, sex, region, deltale, ageatdeath, label=getvaluesPerson(person)
    fromAge,toAge, frequency, options, label=getvaluesRow(person)

    if not fromAge==toAge:
        title=curve.getTitle(result=[0,0,0,0],fromAge=fromAge,toAge=toAge,freq=frequency,cont=1,options=options)
    else:
        title=curve.getTitle(result=[0,0,0,0],fromAge=fromAge,freq=frequency,cont=1,options=options)

    st,en,factor,timeInterval=returnFreq(frequency)
    #The plot
    LxNoI,Lx,Rng=curve.getCurve(options=options,cont=1)

    axes_options={'x':dict(label='Age'),'y':dict(label='Multiplier')}

    pdf_fig=plt.figure(person)
    pdf_fig.title=''
    plt.scales(scales={'x':LinearScale(),'y':LinearScale(allow_padding=False)})
    #scale
    agap=0
    if person=='DEPENDENT': agap=agegap()
    leftX=getLeftX(fatal.value)-agap
    pdf_fig.layout.height='400px'
    pdf_line=plt.plot(Rng,Lx, axes_options=axes_options)
    pdf_line.scales['x'].min=leftX
    pdf_line.scales['x'].max=125-agap
    pdf_line.scales['y'].min=0
    pdf_line.scales['y'].max=2

    pdf_age=plt.vline(age,colors=['green'],labels=['Age at trial'], display_legend=True)
    pdf_aad=plt.vline(ageatdeath,colors=['black'],labels=['Age at death'],display_legend=fatal.value,visible=True)
    mask=(Rng>=fromAge) & (Rng<=toAge)#Boolean mask
    R=Rng[mask]
    L=Lx[mask]

    pdf_fill=plt.plot(R,L,fill='bottom',preserve_domain={'x':True,'y':True},colors=['red'],fill_opacities=[0.5])

    #single point
    y1=np.interp(fromAge,Rng,Lx)
    pdf_start=plt.plot(colors=['red'] ,preserve_domain={'x': True, 'y': False}, x=[fromAge, fromAge], y=[0, y1])
    y2=np.interp(toAge,Rng,Lx)
    pdf_end=plt.plot(colors=['red'] ,preserve_domain={'x': True, 'y': False}, x=[toAge, toAge], y=[0, y2])
        
    display(pdf_fig)
    return pdf_fig, pdf_line, pdf_age, pdf_aad, pdf_fill, pdf_start, pdf_end


In [15]:
pdf={}
pdf['CLAIMANT']=drawFigure('CLAIMANT')
pdf['DEPENDENT']=drawFigure('DEPENDENT')




Figure(axes=[Axis(label='Age', scale=LinearScale(max=125.0, min=29.0), side='bottom'), Axis(label='Multiplier'…

In [11]:
on_value_change(None)
setFatal()

